In [ ]:
import telegram
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io
import pandas as pd
import pandahouse as ph
import datetime as dt
from airflow.decorators import dag, task

In [ ]:
default_args = {
    'owner': 'name',
    'depends_on_past': False,
    'retries': 2,
    'retry_delay': dt.timedelta(minutes=5),
    'start_date': dt.datetime(2022, 7, 30),
}
schedule_interval = '0 11 * * *'
connection = {'host': 'host',
                      'database':'db',
                      'user':'name', 
                      'password':'password'
                     }

In [ ]:
my_token = '<token>'
bot = telegram.Bot(token=my_token)

In [ ]:
q = """
SELECT start_day, day, ("MAX(users)" / initial) as retention FROM
(SELECT * FROM
(SELECT start_day, "MAX(users)" AS initial FROM
(SELECT day AS day,
              start_day AS start_day,
              max(users) AS "MAX(users)"
FROM
  (SELECT toString(start_day) as start_day,
          toString(day) as day,
          count(user_id) AS users
   FROM
     (SELECT *
      FROM
        (SELECT user_id,
                min(toDate(time)) AS start_day
         FROM simulator_20220720.feed_actions
         GROUP BY user_id) t1
      JOIN
        (SELECT DISTINCT user_id,
                         toDate(time) AS day
         FROM simulator_20220720.feed_actions) t2 USING user_id
      WHERE start_day >= today() - 10 )
   GROUP BY start_day,
            day) AS virtual_table
GROUP BY day, start_day) as sub
WHERE day = start_day) as q1
FULL OUTER JOIN
(SELECT day AS day,
              start_day AS start_day,
              max(users) AS "MAX(users)"
FROM
  (SELECT toString(start_day) as start_day,
          toString(day) as day,
          count(user_id) AS users
   FROM
     (SELECT *
      FROM
        (SELECT user_id,
                min(toDate(time)) AS start_day
         FROM simulator_20220720.feed_actions
         GROUP BY user_id) t1
      JOIN
        (SELECT DISTINCT user_id,
                         toDate(time) AS day
         FROM simulator_20220720.feed_actions) t2 USING user_id
      WHERE start_day >= today() - 10 )
   GROUP BY start_day,
            day) AS virtual_table
GROUP BY day,
         start_day) as q2 USING(start_day))
"""
q1 = """
SELECT COUNT(DISTINCT user_id) as users, DATE(time) as day
FROM simulator_20220720.feed_actions
WHERE DATE(time) BETWEEN DATE(today()-2) and DATE(yesterday())
GROUP BY DATE(time)
ORDER BY day
"""
q2 = """
SELECT COUNT(DISTINCT user_id) as users, DATE(time) as day
FROM simulator_20220720.message_actions
WHERE DATE(time) BETWEEN DATE(today()-2) and DATE(yesterday())
GROUP BY DATE(time)
ORDER BY day
"""
q3 = """
SELECT COUNT(DISTINCT user_id) as users, day FROM
(SELECT * FROM
(SELECT DISTINCT user_id as user_id, DATE(time) as day
FROM simulator_20220720.feed_actions
WHERE DATE(time) BETWEEN DATE(today()-2) and DATE(yesterday())) as q1
INNER JOIN
(SELECT DISTINCT user_id as user_id, DATE(time) as day
FROM simulator_20220720.message_actions
WHERE DATE(time) BETWEEN DATE(today()-2) and DATE(yesterday())) as q2 ON q1.user_id=q2.user_id AND q1.day=q2.day)
GROUP BY day
ORDER BY day
"""
q4 = """
SELECT previous_week AS previous_week,
       status AS status,
       max(num_users) AS users
FROM
  (SELECT this_week,
          previous_week, -uniq(user_id) as num_users,
                          status
   FROM
     (SELECT user_id,
             groupUniqArray(toMonday(toDate(time))) as weeks_visited,
             addWeeks(arrayJoin(weeks_visited), +1) this_week,
             if(has(weeks_visited, this_week) = 1, 'retained', 'gone') as status,
             addWeeks(this_week, -1) as previous_week
      FROM simulator_20220720.feed_actions
      group by user_id)
   where status = 'gone'
   group by this_week,
            previous_week,
            status
   HAVING this_week != addWeeks(toMonday(today()), +1)
   union all SELECT this_week,
                    previous_week,
                    toInt64(uniq(user_id)) as num_users,
                    status
   FROM
     (SELECT user_id,
             groupUniqArray(toMonday(toDate(time))) as weeks_visited,
             arrayJoin(weeks_visited) this_week,
             if(has(weeks_visited, addWeeks(this_week, -1)) = 1, 'retained', 'new') as status,
             addWeeks(this_week, -1) as previous_week
      FROM simulator_20220720.feed_actions
      group by user_id)
   group by this_week,
            previous_week,
            status) AS virtual_table
GROUP BY previous_week,
         status
ORDER BY users DESC
"""

In [ ]:
yesterday = dt.date.today() - dt.timedelta(days=1)
to_days_ago = dt.date.today() - dt.timedelta(days=2)

In [ ]:
def send_message(chat_id='<id>'):
    actions = ph.read_clickhouse(q1, connection=connection)
    messages = ph.read_clickhouse(q2, connection=connection)
    both = ph.read_clickhouse(q3, connection=connection)
    msg = 'Показатели за {} c динамикой к {}\nКоличество уникальных пользователей в ленте новостей: {:_.0f} ({})\nКоличество пользователей в мессенджере: {:_.0f} ({})\nКоличество пользователей, воспользовавшихся и лентой новостей, и мессенджером: {:_.0f} ({})'.format(yesterday.strftime('%d.%m.%Y'), to_days_ago.strftime('%d.%m.%Y'), actions['users'][1], (int(actions['users'][1]) - int(actions['users'][0])), messages['users'][1], int(messages['users'][1]) - int(messages['users'][0]), both['users'][1], int(both['users'][1]) - int(both['users'][0])).replace('_', ' ')
    bot.sendMessage(chat_id=chat_id, text=msg)
def send_retention(chat_id='<id>'): 
    df = ph.read_clickhouse(q, connection=connection)
    df = pd.pivot_table(data=df, index='start_day', columns='day', values='retention', aggfunc='max')
    plt.figure(figsize=(10, 10))
    plt.title('Retention Rate')
    sns.heatmap(df, annot=True, fmt='.0%');
    plot_object = io.BytesIO()
    plt.savefig(plot_object)
    plot_object.seek(0)
    plot_object.name = 'RetentionRate.png'
    plt.close()
    bot.sendPhoto(chat_id=chat_id, photo=plot_object)
def send_retention_status(chat_id='<id>'):    
    df = ph.read_clickhouse(q4, connection=connection)
    df['week'] = df['previous_week'].apply(lambda x: x.strftime(format='%d.%m.%Y'))
    fig, ax = plt.subplots(figsize=(10, 10))
    plt.title('Retention status')
    ax.tick_params(axis='x', labelrotation=45)
    sns.barplot(data=df.sort_values(by='previous_week'), hue='status', x='week', y='users')
    plot_object = io.BytesIO()
    plt.savefig(plot_object)
    plot_object.seek(0)
    plot_object.name = 'RetentionRate.png'
    plt.close()
    bot.sendPhoto(chat_id=chat_id, photo=plot_object)

In [ ]:
@dag(default_args=default_args, schedule_interval=schedule_interval, catchup=False)
def lmk_analitic_report2():
    @task()
    def make_report():
        send_message(chat_id='<id>')
        send_retention(chat_id='<id>')
        send_retention_status(chat_id='<id>')
    make_report()

In [ ]:
lmk_analitic_report2 = lmk_analitic_report2()